In [1]:
!pip install langchain-community langchain-text-splitters  pypdf  langchain-huggingface sentence-transformers faiss-cpu ollama langchain-ollama   --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 85.0 MB/s eta 0:00:00


In [2]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 20.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 31.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
model_id = "Qwen/Qwen3-4B-Instruct-2507"

# ==========================================
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 確保 pipeline 正常運作的常見修復
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# ==========================================
# 建立原生 Transformers Pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,      # 決定回答長度
    temperature=0.2,         # RAG 建議設低一點，減少幻覺
    repetition_penalty=1.1,  # 避免鬼打牆
    return_full_text=False   # 重要：設為 False 只回傳生成的部分
)

# . 轉為 LangChain 的 LLM 物件
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# (關鍵) 轉為 ChatModel 物件
# ChatHuggingFace 會自動處理 prompt template (User/System/Assistant 格式)
chat_model = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

Passing `generation_config` together with generation-related arguments=({'repetition_penalty', 'temperature', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.


In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

##Define the explainable retriever class

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
class ExplainableRetriever:
    def __init__(self,texts,embedding,chat_moedel):
        self.embedding=embedding
        self.llm=chat_moedel
        self.vectorstore=FAISS.from_texts(texts,self.embedding)
        self.retriever=self.vectorstore.as_retriever()

        explain_prompt=PromptTemplate(
            input_variables=["query","context"],
            template="""
            Analyze the relationship between the following query and the retrieved context.
            Explain why this context is relevant to the query and how it might help answer the query.

            Query: {query}

            Context: {context}

            Explanation:
            """
        )

        self.explain_chain=explain_prompt | self.llm

    def retrieve_and_explian(self,query):
        docs=self.retriever.invoke(query,k=3)
        explained_result=[]

        for doc in docs:
            input_data={"query":query,"context":doc.page_content}
            explanation=self.explain_chain.invoke(input_data)
            explained_result.append({
                "content":doc.page_content,
                "explanation":explanation
            })

        return explained_result


##Create a mock example and explainable retriever instance

In [15]:
# Usage
texts = [
    "The sky is blue because of the way sunlight interacts with the atmosphere.",
    "Photosynthesis is the process by which plants use sunlight to produce energy.",
    "Global warming is caused by the increase of greenhouse gases in Earth's atmosphere."
]

explainable_retriever = ExplainableRetriever(texts,embedding_model,chat_model)

##Show the results

In [16]:
query = "Why is the sky blue?"
results = explainable_retriever.retrieve_and_explian(query)

for i, result in enumerate(results, 1):
    print(f"Result {i}:")
    print(f"Content: {result['content']}")
    print(f"Explanation: {result['explanation']}")
    print()

Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=128) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Result 1:
Content: The sky is blue because of the way sunlight interacts with the atmosphere.
Explanation: content='The context is highly relevant to the query "Why is the sky blue?" because it directly addresses the scientific explanation behind the phenomenon. The statement "The sky is blue because of the way sunlight interacts with the atmosphere" provides a foundational, accurate principle that explains the observed color of the sky.\n\nThis context helps answer the query by introducing the key concept—sunlight scattering in the atmosphere—which is essential for understanding the actual mechanism (Rayleigh scattering). While the context does not go into detailed physics or wavelengths, it correctly identifies the core cause and sets the stage for further exploration. Therefore, it serves as an appropriate and concise starting point for answering' additional_kwargs={} response_metadata={} id='lc_run--019c4c71-947d-7980-b08c-4c19eba1fbe1-0' tool_calls=[] invalid_tool_calls=[]

Result